## install Python SDKs

In [2]:
import sys

In [3]:
!{sys.executable} -m pip install sagemaker-experiments

    100% |████████████████████████████████| 133kB 29.5MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 8.3MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 31.9MB/s ta 0:00:01
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.14.0 which is incompatible.
awscli 1.16.283 has requirement s3transfer<0.3.0,>=0.2.0, but you'll have s3transfer 0.3.0 which is incompatible.
  Found existing installation: botocore 1.13.19
    Uninstalling botocore-1.13.19:
      Successfully uninstalled botocore-1.13.19
  Found existing installation: s3transfer 0.2.1
    Uninstalling s3transfer-0.2.1:
      Successfully uninstalled s3transfer-0.2.1
  Found existing installation: boto3 1.10.19
    Uninstalling boto3-1.10.19:
      Successfully uninstalled boto3-1.10.19
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --u

In [5]:
!{sys.executable} -m pip install awscli --upgrade
!{sys.executable} -m pip install boto3 --upgrade

    100% |████████████████████████████████| 2.8MB 14.4MB/s ta 0:00:01
Requirement not upgraded as not directly required: s3transfer<0.4.0,>=0.3.0 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from awscli) (0.3.0)
Requirement not upgraded as not directly required: PyYAML<5.3,>=3.10 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from awscli) (3.12)
Requirement not upgraded as not directly required: colorama<0.4.2,>=0.2.5 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from awscli) (0.3.9)
Requirement not upgraded as not directly required: botocore==1.14.0 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from awscli) (1.14.0)
Requirement not upgraded as not directly required: docutils<0.16,>=0.10 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from awscli) (0.14)
Requirement not upgraded as not directly required: rsa<=3.5.0,>=3.1.2 in /home/ec2-user/anaconda3

In [1]:
import time

import boto3
import pandas as pd

import sagemaker
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [2]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

Track dataset

In [3]:
with Tracker.create(display_name="dataset", sagemaker_boto_client=sm) as tracker:
    tracker.log_input(name="omniglot-dataset", 
                      media_type="s3/uri", value='s3://test-meta-learning-data/data/omniglot/')

### Create experiment

In [4]:
protonet_experiment = Experiment.create(
    experiment_name=f'protonet-omniglot-dummy-exp-{int(time.time())}',
    description='trying out experiment functionality with protonet',
    sagemaker_boto_client=sm)
print(protonet_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f7dd9d07320>,experiment_name='protonet-omniglot-dummy-exp-1578922953',description='trying out experiment functionality with protonet',experiment_arn='arn:aws:sagemaker:eu-west-1:175716149256:experiment/protonet-omniglot-dummy-exp-1578922953',response_metadata={'RequestId': 'af8fbf2f-c37c-4258-a1d6-ef411ebd7db0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'af8fbf2f-c37c-4258-a1d6-ef411ebd7db0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Mon, 13 Jan 2020 13:42:33 GMT'}, 'RetryAttempts': 0})


In [5]:
preprocessing_trial_component = tracker.trial_component

In [6]:
protonet_trial = Trial.create(
    trial_name='dummy-bla-2',
    experiment_name=protonet_experiment.experiment_name,
    sagemaker_boto_client=sm)

protonet_trial.add_trial_component(preprocessing_trial_component)

In [7]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='proto_experiment.py',
                    source_dir='../few_shot/',
                    role=role,
                    sagemaker_session=sagemaker.Session(sagemaker_client=sm),
                    framework_version='1.2.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.2xlarge',
                    hyperparameters={
                        'epochs':10
                    },
                    metric_definitions=[
                        {'Name':'train:loss', 'Regex':'loss: (.*?);'},
                        {'Name':'epoch', 'Regex':'epoch: (.*?),'}
                    ],
                    enable_sagemaker_metrics=True
                )

In [8]:
estimator.fit(inputs={'train':'s3://test-meta-learning-data/data/omniglot/images_background.zip',
                      'test':'s3://test-meta-learning-data/data/omniglot/images_evaluation.zip'},
              job_name=f'test-{int(time.time())}',
              experiment_config={
                  'TrialName': protonet_trial.trial_name,
                  "TrialComponentDisplayName": "Training"
              }
            )

INFO:sagemaker:Creating training-job with name: test-1578923113


2020-01-13 13:45:14 Starting - Starting the training job...
2020-01-13 13:45:15 Starting - Launching requested ML instances......
2020-01-13 13:46:40 Starting - Preparing the instances for training......
2020-01-13 13:47:31 Downloading - Downloading input data...
2020-01-13 13:48:04 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-13 13:48:05,834 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-13 13:48:05,836 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-13 13:48:05,847 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-01-13 13:48:08,897 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-01-13 13:48:09,240 sagemaker-containers INFO     Module proto_experiment does not provide a setup.py. 
Gener

In [9]:
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name=protonet_experiment.experiment_name
)

In [10]:
analytic_table = trial_component_analytics.dataframe()

In [11]:
analytic_table

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,epochs,sagemaker_container_log_level,sagemaker_enable_cloudwatch_metrics,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory
0,test-1578923113-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:175716149256:train...,763104351884.dkr.ecr.eu-west-1.amazonaws.com/p...,1.0,ml.c4.2xlarge,30.0,10.0,20.0,false,"""test-1578923113""","""proto_experiment.py""","""eu-west-1""","""s3://sagemaker-eu-west-1-175716149256/test-15..."
1,TrialComponent-2020-01-13-134221-hjcl,dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
analytic_table

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,epochs,sagemaker_container_log_level,sagemaker_enable_cloudwatch_metrics,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory
0,test-1578917308-aws-training-job,test-1578917308-aws-training-job,arn:aws:sagemaker:eu-west-1:175716149256:train...,763104351884.dkr.ecr.eu-west-1.amazonaws.com/p...,1.0,ml.c4.2xlarge,30.0,10.0,20.0,false,"""test-1578917308""","""proto_experiment.py""","""eu-west-1""","""s3://sagemaker-eu-west-1-175716149256/test-15..."
1,test-1578917025-aws-training-job,test-1578917025-aws-training-job,arn:aws:sagemaker:eu-west-1:175716149256:train...,763104351884.dkr.ecr.eu-west-1.amazonaws.com/p...,1.0,ml.c4.2xlarge,30.0,10.0,20.0,false,"""test-1578917025""","""proto_experiment.py""","""eu-west-1""","""s3://sagemaker-eu-west-1-175716149256/test-15..."
2,TrialComponent-2020-01-13-115108-aqxt,dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
